### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
tmdb_access_key = os.getenv("TMDB_ACCESS_KEY")



### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"

filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&begin_date={begin_date}&end_date={end_date}&sort={sort}&fl={field_list}"

print(query_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=Ab7tjs5wfqjJJtfsDF3MU6G3lef0HW3O&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&begin_date=20130101&end_date=20230531&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count


In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 19):
    # create query with a page number
    # API results show 10 articles at a time
    page_query_url = f"{query_url}&page={str(page)}"
        
    # Make a "GET" request and retrieve the JSON
    articles = requests.get(page_query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for article in articles["response"]["docs"]:
            reviews_list.append(article)
        # Print the page that was just retrieved
            print(f"Preview of Review: {article['snippet']}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"Page {page} has no results.")
        break
    

Preview of Review: A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.
Preview of Review: Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.
Preview of Review: Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.
Preview of Review: Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.
Preview of Review: A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.
Preview of Review: A film adaptation of Taylor Jenkins Reid’s novel has potential for drama, but it stumbles on stock melodrama.
Preview of Review: There’s not much Lennon music heard in this doc about his affair with May Pang, and given how much Pang trashes his wife, Yoko Ono, it’s no surprise it was withheld.

In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
nyt_top5_preview = []
for top5 in range(0, 4):
    article = reviews_list[top5]
    nyt_top5_preview.append(article)
json_top5 = json.dumps(nyt_top5_preview, indent=4)
print(json_top5)




[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_df = pd.json_normalize(reviews_list)
reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,https://www.nytimes.com/2017/06/22/movies/my-j...,"In “My Journey Through French Cinema,” Mr. Tav...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2017-06-22T21:14:56+0000,773,"Review: Those Movies, Himself — Bertrand Taver...",None,None,Reflecting on a Lifelong Love,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
186,https://www.nytimes.com/2017/06/22/movies/the-...,Kumail Nanjiani stars opposite Zoe Kazan in th...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-06-22T09:00:31+0000,1073,"Review: In ‘The Big Sick,’ Comedy Is Hard, Lov...",None,None,Love and (Almost) Death,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
187,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,"[{'name': 'creative_works', 'value': 'Lost in ...",2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
188,https://www.nytimes.com/2017/06/01/movies/vinc...,This combination crime drama and romance finds...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-06-01T19:52:51+0000,287,"Review: In ‘Vincent N Roxxy,’ Love Is in the A...",None,None,Vincent N Roxxy,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

reviews_list_df["title"] = reviews_list_df["headline.main"].apply( lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")] )

reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,https://www.nytimes.com/2017/06/22/movies/my-j...,"In “My Journey Through French Cinema,” Mr. Tav...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2017-06-22T21:14:56+0000,773,"Review: Those Movies, Himself — Bertrand Taver...",None,None,Reflecting on a Lifelong Love,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"Review: Those Movies, Himself — Bertrand Taver..."
186,https://www.nytimes.com/2017/06/22/movies/the-...,Kumail Nanjiani stars opposite Zoe Kazan in th...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-06-22T09:00:31+0000,1073,"Review: In ‘The Big Sick,’ Comedy Is Hard, Lov...",None,None,Love and (Almost) Death,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"The Big Sick,’ Comedy Is Hard, Love Harde"
187,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,"[{'name': 'creative_works', 'value': 'Lost in ...",2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,Lost in Paris
188,https://www.nytimes.com/2017/06/01/movies/vinc...,This combination crime drama and romance finds...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-06-01T19:52:51+0000,287,"Review: In ‘Vincent N Roxxy,’ Love Is in the A...",None,None,Vincent N Roxxy,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,"Vincent N Roxxy

In [8]:

# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        print(keyword)
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_list_df["keywords"]= reviews_list_df["keywords"].apply(extract_keywords)




subject: Movies;
creative_works: The Attachment Diaries (Movie);
persons: Diment, Valentin Javier;
subject: Movies;
persons: Kapur, Shekhar;
persons: James, Lily;
persons: Azmi, Shabana;
persons: Thompson, Emma;
subject: Movies;
creative_works: You Can Live Forever (Movie);
persons: Slutsky, Mark;
persons: Watts, Sarah (Film Director);
persons: O'Driscoll, Anwen;
persons: Laporte, June (Actor);
subject: Movies;
creative_works: A Tourist's Guide to Love (Movie);
persons: Tsuchida, Steven;
persons: Cook, Rachael Leigh;
subject: Movies;
persons: Zlotowski, Rebecca;
creative_works: Other People's Children (Movie);
subject: Movies;
persons: Bracey, Luke (1989- );
persons: Liu, Simu;
persons: Reid, Taylor Jenkins;
persons: Soo, Phillipa;
persons: Fickman, Andy;
creative_works: One True Loves (Movie);
subject: Documentary Films and Programs;
creative_works: The Lost Weekend: A Love Story (Movie);
persons: Pang, May;
persons: Lennon, John;
persons: Ono, Yoko;
persons: Brandstein, Eve;
persons:

In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = reviews_list_df["title"].to_list()
title_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
headers = {
    "accept": "application/json",
    "Authorization": "Bearer " + tmdb_access_key
}



In [11]:
# Create an empty list to store the results
movie_list = []
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1


# Loop through the titles
for title in title_list:
    
    # Check if we need to sleep before making a request
    if request_counter == 50:
        time.sleep(1)
        request_counter = 1
        print("Application is sleeping.")

    # Add 1 to the request counter
    else:
        request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    request_url = url + title
    response_data = requests.get(request_url, headers=headers)
    data = response_data.json()
    

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    
    # empty string and lists
    movie_id = ""
    genre_names = []
    spoken_languages = []
    production_countries_list = []
    details_response = ""
    try:
        # Get movie id
        # tmdb_movies_list = title_list
        movie_id = str(data["results"][0]["id"])
        

    except IndexError:
        print("IndexError; Movie id was not found for " + title )
        

    except:
        print("title not found: " + title)
        pass

    try:
        details_url = "https://api.themoviedb.org/3/movie/" + movie_id

        # Execute "GET" request with url
        details_data = requests.get(details_url, headers=headers)
        details_response = details_data.json()
        
        
        # Extract the genre names into a list
        genres = details_response["genres"]
        
        
        for genre in genres:
            genre_names.append(genre["name"])
        
        
    except IndexError:
        print("IndexError; Movie details or genres not found " + title )
        pass

    except:
        print("title not found: " + title)
        pass

    try:
        # Extract the spoken_languages' English name into a list
        languages = details_response["spoken_languages"]
        print(languages)
        for language in languages:
            spoken_languages.append(language["english_name"])
        

    except IndexError:
        print("IndexError; spoken languages not found " + title )
        break

    except:
        print("Lanaguage -- title not found: " + title)
        pass

    try:

        # Extract the production_countries' name into a list
        production_countries = details_response["production_countries"]
        
        for production_country in production_countries:
            production_countries_list.append(production_country["name"])
        

    except IndexError:
        print("IndexError; production countries not found for " + title )
        

    except:
        print("Production -- title not found: " + title)
        pass  

    try:

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_dict = {}
        tmdb_movies_dict = {"movie_id": movie_id, "title": title, "genres":genre_names, "languages":spoken_languages, "production_country": production_countries_list}
        tmdb_movies_list.append(tmdb_movies_dict)

        # Print out the title that was found
        print("Found Movie: " + title)

    except IndexError:
        print("IndexError; Movie details not found " + title )
        pass

    except:
        print("Title not found: " + title)
        pass


[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'name': 'Mystery'}, {'id': 53, 'name': 'Thriller'}, {'id': 27, 'name': 'Horror'}]
['Drama', 'Mystery', 'Thriller', 'Horror']
[{'english_name': 'Spanish', 'iso_639_1': 'es', 'name': 'Español'}]
['Spanish']
[{'iso_3166_1': 'AR', 'name': 'Argentina'}]
['Argentina']
Found Movie: The Attachment Diaries
IndexError; Movie id was not found for What’s Love Got to Do With It?’ Probably a Lo
title not found: What’s Love Got to Do With It?’ Probably a Lo
Lanaguage -- title not found: What’s Love Got to Do With It?’ Probably a Lo
Production -- title not found: What’s Love Got to Do With It?’ Probably a Lo
Found Movie: What’s Love Got to Do With It?’ Probably a Lo
[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]
['Drama', 'Romance']
[{'english_name': 'English', 'iso_639_1': 'en', 'name': 'English'}, {'english_name': 'French', 'iso_639_1': 'fr', 'name': 'Français'}]
['English', 'French']
[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 

In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

movie_top5_preview = []
for top5 in range(0, 4):
    title = tmdb_movies_list[top5]

    movie_top5_preview.append(title)
json_top5 = json.dumps(movie_top5_preview, indent=4)
print(json_top5)


[
    {
        "movie_id": "743040",
        "title": "The Attachment Diaries",
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "languages": [
            "Spanish"
        ],
        "production_country": [
            "Argentina"
        ]
    },
    {
        "movie_id": "",
        "title": "What\u2019s Love Got to Do With It?\u2019 Probably a Lo",
        "genres": [],
        "languages": [],
        "production_country": []
    },
    {
        "movie_id": "887580",
        "title": "You Can Live Forever",
        "genres": [
            "Drama",
            "Romance"
        ],
        "languages": [
            "English",
            "French"
        ],
        "production_country": [
            "Canada",
            "United States of America"
        ]
    },
    {
        "movie_id": "813726",
        "title": "A Tourist\u2019s Guide to Love",
        "genres": [
            "Romance",
        

In [13]:
# Convert the results to a DataFrame
tmdb_movies_df= pd.json_normalize(tmdb_movies_list)
tmdb_movies_df

,movie_id,title,genres,languages,production_country
0,743040,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,,What’s Love Got to Do With It?’ Probably a Lo,[],[],[]
2,887580,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
3,813726,A Tourist’s Guide to Love,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,803694,Other People’s Children,"[Drama, Comedy]","[French, English]",[France]
...,...,...,...,...,...
185,,"Review: Those Movies, Himself — Bertrand Taver...",[],[],[]
186,,"The Big Sick,’ Comedy Is Hard, Love Harde",[],[],[]
187,413778,Lost in Paris,[Comedy],[French],"[Belgium, France]"
188,,"Vincent N Roxxy,’ Love Is in the Air, N So Is Fea",[],[],[]


### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title
combined_df = pd.merge(reviews_list_df, tmdb_movies_df, on= "title", how="inner")
combined_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,movie_id,genres,languages,production_country
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo,,[],[],[]
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,887580,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love,813726,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children,803694,"[Drama, Comedy]","[French, English]",[France]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,https://www.nytimes.com/2017/06/22/movies/my-j...,"In “My Journey Through French Cinema,” Mr. Tav...",The New York Times,subject: Documentary Films and Programs;person...,2017-06-22T21:14:56+0000,773,"Review: Those Movies, Himself — Bertrand Taver...",None,None,Reflecting on a Lifelong Love,...,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"Review: Those Movies, Himself — Bertrand Taver...",,[],[],[]
186,https://www.nytimes.com/2017/06/22/movies/the-...,Kumail Nanjiani stars opposite Zoe Kazan in th...,The New York Times,subject: Movies;creative_works: The Big Sick (...,2017-06-22T09:00:31+0000,1073,"Review: In ‘The Big Sick,’ Comedy Is Hard, Lov...",None,None,Love and (Almost) Death,...,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"The Big Sick,’ Comedy Is Hard, Love Harde",,[],[],[]
187,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,Lost in Paris,413778,[Comedy],[French],"[Belgium, France]"
1

In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_tofix = ["genres", "languages", "production_country", "headline.main", "byline.person"]
print(columns_tofix)

# Create a list of characters to remove
replacements = ["[", "]", "{", "}"]

# Loop through the list of columns to fix
for column in columns_tofix:
    print(column)
    combined_df
    # Convert the column to type 'str'
    combined_df[column] = combined_df[column].astype(str)
    # print(combined_df[column])
    
    # Loop through characters to remove
    for char in replacements:
        combined_df[column] = combined_df[column].str.replace(char, "")
        # column_string = column.replace(char, to_remove)
            

# Display the fixed DataFrame
combined_df

['genres', 'languages', 'production_country', 'headline.main', 'byline.person']
genres
languages
production_country
headline.main
byline.person


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,movie_id,genres,languages,production_country
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"'firstname': 'Jeannette', 'middlename': None, ...",None,The Attachment Diaries,743040,"'Drama', 'Mystery', 'Thriller', 'Horror'",'Spanish','Argentina'
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,None,None,By Jeannette Catsoulis,"'firstname': 'Jeannette', 'middlename': None, ...",None,What’s Love Got to Do With It?’ Probably a Lo,,,,
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,None,None,By Elisabeth Vincentelli,"'firstname': 'Elisabeth', 'middlename': None, ...",None,You Can Live Forever,887580,"'Drama', 'Romance'","'English', 'French'","'Canada', 'United States of America'"
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,None,None,By Elisabeth Vincentelli,"'firstname': 'Elisabeth', 'middlename': None, ...",None,A Tourist’s Guide to Love,813726,"'Romance', 'Comedy'","'English', 'Vietnamese'",'United States of America'
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,None,None,By Manohla Dargis,"'firstname': 'Manohla', 'middlename': None, 'l...",None,Other People’s Children,803694,"'Drama', 'Comedy'","'French', 'English'",'France'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,https://www.nytimes.com/2017/06/22/movies/my-j...,"In “My Journey Through French Cinema,” Mr. Tav...",The New York Times,subject: Documentary Films and Programs;person...,2017-06-22T21:14:56+0000,773,"Review: Those Movies, Himself — Bertrand Taver...",None,None,Reflecting on a Lifelong Love,...,None,None,By Manohla Dargis,"'firstname': 'Manohla', 'middlename': None, 'l...",None,"Review: Those Movies, Himself — Bertrand Taver...",,,,
186,https://www.nytimes.com/2017/06/22/movies/the-...,Kumail Nanjiani stars opposite Zoe Kazan in th...,The New York Times,subject: Movies;creative_works: The Big Sick (...,2017-06-22T09:00:31+0000,1073,"Review: In ‘The Big Sick,’ Comedy Is Hard, Lov...",None,None,Love and (Almost) Death,...,None,None,By Manohla Dargis,"'firstname': 'Manohla', 'middlename': None, 'l...",None,"The Big Sick,’ Comedy Is Hard, Love Harde",,,,
187,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,None,None,By Ben Kenigsberg,"'firstname': 'Ben', 'middlename': None, 'lastn...",None,Lost in Paris,413778,'Comedy','French',"'Belgium', 'France

In [16]:
# Drop "byline.person" column
combined_df = combined_df.drop("byline.person", axis=1)

In [17]:
# Delete duplicate rows and reset index
combined_df = combined_df.drop_duplicates(keep = False)
combined_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,title,movie_id,genres,languages,production_country
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,743040,"'Drama', 'Mystery', 'Thriller', 'Horror'",'Spanish','Argentina'
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What’s Love Got to Do With It?’ Probably a Lo,,,,
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None,You Can Live Forever,887580,"'Drama', 'Romance'","'English', 'French'","'Canada', 'United States of America'"
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None,A Tourist’s Guide to Love,813726,"'Romance', 'Comedy'","'English', 'Vietnamese'",'United States of America'
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None,Other People’s Children,803694,"'Drama', 'Comedy'","'French', 'English'",'France'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,https://www.nytimes.com/2017/06/22/movies/my-j...,"In “My Journey Through French Cinema,” Mr. Tav...",The New York Times,subject: Documentary Films and Programs;person...,2017-06-22T21:14:56+0000,773,"Review: Those Movies, Himself — Bertrand Taver...",None,None,Reflecting on a Lifelong Love,None,None,None,By Manohla Dargis,None,"Review: Those Movies, Himself — Bertrand Taver...",,,,
186,https://www.nytimes.com/2017/06/22/movies/the-...,Kumail Nanjiani stars opposite Zoe Kazan in th...,The New York Times,subject: Movies;creative_works: The Big Sick (...,2017-06-22T09:00:31+0000,1073,"Review: In ‘The Big Sick,’ Comedy Is Hard, Lov...",None,None,Love and (Almost) Death,None,None,None,By Manohla Dargis,None,"The Big Sick,’ Comedy Is Hard, Love Harde",,,,
187,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,None,Lost in Paris,413778,'Comedy','French',"'Belgium', 'France'"
188,https://www.nytimes.com/2017/06/01/movies/vinc...,This combination crime drama and romance finds...,The New York Times,subject: Movies;creative_works: Vincent N Roxx...,2017-06-01T19:52:51+0000,287,"Review: In ‘Vincent N Roxxy,’ Love Is in the A...",None,None,Vincent N Roxxy,None,None,None,By Ken Jaworowski,None,"Vincent N Roxxy,’ Love Is in the Air, N So Is Fea",,,,


In [18]:
# Export data to CSV without the index
combined_df.to_csv("Movie_output_from_combined_API_data.csv", index=False)